In [136]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

# Load data
sentences_df = pd.read_csv('/content/drive/MyDrive/New folder/training/sentences.tsv', sep='\t')
labels_df = pd.read_csv('/content/drive/MyDrive/New folder/training/labels.tsv', sep='\t')

In [138]:
sentences_df

,Text-ID,Sentence-ID,Text
0,BG_002,1,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ..."
1,BG_002,2,КВ.
2,BG_002,3,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...
3,BG_002,4,Вече месеци наред той е обект на проверка за к...
4,BG_002,5,Там от години прокуратурата се явява съдружник...
...,...,...,...
40662,TR_M_022,12,Kolay kazanç sağlayan ve istihdam öngörmeyen r...
40663,TR_M_022,13,"Vergi tabana yayılacak, yoksul kesimin vergi y..."
40664,TR_M_022,14,Sosyal refahı artıracak destekleyici düzenleme...
40665,TR_M_022,15,Yoksulluktan en fazla etkilenen eğitim düzeyi ...


In [139]:
labels_df

,Text-ID,Sentence-ID,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,Hedonism constrained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,BG_002,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BG_002,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BG_002,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BG_002,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BG_002,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,TR_M_022,12,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,TR_M_022,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,TR_M_022,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,TR_M_022,15,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
df1 = sentences_df
df2 = labels_df

In [141]:
import pandas as pd
merged_df = pd.merge(df1, df2, on=['Text-ID', 'Sentence-ID'])
merged_df


,Text-ID,Sentence-ID,Text,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,BG_002,1,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BG_002,2,КВ.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BG_002,3,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BG_002,4,Вече месеци наред той е обект на проверка за к...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BG_002,5,Там от години прокуратурата се явява съдружник...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,TR_M_022,12,Kolay kazanç sağlayan ve istihdam öngörmeyen r...,0.0,0.0,0.0,0.0,0.5,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,TR_M_022,13,"Vergi tabana yayılacak, yoksul kesimin vergi y...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,TR_M_022,14,Sosyal refahı artıracak destekleyici düzenleme...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,TR_M_022,15,Yoksulluktan en fazla etkilenen eğitim düzeyi ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
merged_df = merged_df[merged_df['Text-ID'].str.contains('EN')]

In [143]:
merged_df

,Text-ID,Sentence-ID,Text,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,BG_002,1,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BG_002,2,КВ.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BG_002,3,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BG_002,4,Вече месеци наред той е обект на проверка за к...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BG_002,5,Там от години прокуратурата се явява съдружник...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,TR_M_022,12,Kolay kazanç sağlayan ve istihdam öngörmeyen r...,0.0,0.0,0.0,0.0,0.5,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,TR_M_022,13,"Vergi tabana yayılacak, yoksul kesimin vergi y...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,TR_M_022,14,Sosyal refahı artıracak destekleyici düzenleme...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,TR_M_022,15,Yoksulluktan en fazla etkilenen eğitim düzeyi ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
merged_df = merged_df[~merged_df.isin(['0.5', '', pd.NA]).any(axis=1)]
merged_df

,Text-ID,Sentence-ID,Text,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,BG_002,1,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BG_002,2,КВ.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BG_002,3,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BG_002,4,Вече месеци наред той е обект на проверка за к...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BG_002,5,Там от години прокуратурата се явява съдружник...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,TR_M_022,12,Kolay kazanç sağlayan ve istihdam öngörmeyen r...,0.0,0.0,0.0,0.0,0.5,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,TR_M_022,13,"Vergi tabana yayılacak, yoksul kesimin vergi y...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,TR_M_022,14,Sosyal refahı artıracak destekleyici düzenleme...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,TR_M_022,15,Yoksulluktan en fazla etkilenen eğitim düzeyi ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
merged_df = merged_df.drop(columns=merged_df.columns[:2])
merged_df

,Text,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,Hedonism constrained,Achievement attained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,КВ.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Вече месеци наред той е обект на проверка за к...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Там от години прокуратурата се явява съдружник...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,Kolay kazanç sağlayan ve istihdam öngörmeyen r...,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,"Vergi tabana yayılacak, yoksul kesimin vergi y...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,Sosyal refahı artıracak destekleyici düzenleme...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,Yoksulluktan en fazla etkilenen eğitim düzeyi ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
X = merged_df.iloc[:, :1]  # Assuming the first two columns are features
Y = merged_df.iloc[:, 1:]  # Assuming the rest of the columns are labels


In [147]:
Y

,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,Hedonism constrained,Achievement attained,Achievement constrained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
df = labels_df

In [149]:
df = df.iloc[:, 2:]

In [150]:
vals = df.values

In [151]:
vals[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [152]:
column_names = [
    "Self-direction: thought",
    "Self-direction: action",
    "Stimulation",
    "Hedonism",
    "Achievement",
    "Power: dominance",
    "Power: resources",
    "Face",
    "Security: personal",
    "Security: societal",
    "Tradition",
    "Conformity: rules",
    "Conformity: interpersonal",
    "Humility",
    "Benevolence: caring",
    "Benevolence: dependability",
    "Universalism: concern",
    "Universalism: nature",
    "Universalism: tolerance"
]

index_to_column = {index: column_name for index, column_name in enumerate(column_names)}
print("Original mapping:")
print(index_to_column)

# Reverse the mapping
column_to_index = {v: k for k, v in index_to_column.items()}
print("\nReversed mapping:")
print(column_to_index)


Original mapping:
{0: 'Self-direction: thought', 1: 'Self-direction: action', 2: 'Stimulation', 3: 'Hedonism', 4: 'Achievement', 5: 'Power: dominance', 6: 'Power: resources', 7: 'Face', 8: 'Security: personal', 9: 'Security: societal', 10: 'Tradition', 11: 'Conformity: rules', 12: 'Conformity: interpersonal', 13: 'Humility', 14: 'Benevolence: caring', 15: 'Benevolence: dependability', 16: 'Universalism: concern', 17: 'Universalism: nature', 18: 'Universalism: tolerance'}

Reversed mapping:
{'Self-direction: thought': 0, 'Self-direction: action': 1, 'Stimulation': 2, 'Hedonism': 3, 'Achievement': 4, 'Power: dominance': 5, 'Power: resources': 6, 'Face': 7, 'Security: personal': 8, 'Security: societal': 9, 'Tradition': 10, 'Conformity: rules': 11, 'Conformity: interpersonal': 12, 'Humility': 13, 'Benevolence: caring': 14, 'Benevolence: dependability': 15, 'Universalism: concern': 16, 'Universalism: nature': 17, 'Universalism: tolerance': 18}


In [153]:
# Define the column names
column_names = [
    "Self-direction: thought",
    "Self-direction: action",
    "Stimulation",
    "Hedonism",
    "Achievement",
    "Power: dominance",
    "Power: resources",
    "Face",
    "Security: personal",
    "Security: societal",
    "Tradition",
    "Conformity: rules",
    "Conformity: interpersonal",
    "Humility",
    "Benevolence: caring",
    "Benevolence: dependability",
    "Universalism: concern",
    "Universalism: nature",
    "Universalism: tolerance"
]

# Create a dictionary to map categories to column names
category_mapping = {
    "Openness to change": column_names[:4],
    "Self-enhancement": column_names[4:8],
    "Conservation": column_names[8:13],
    "Self-transcendence": column_names[13:]
}

print(category_mapping)


{'Openness to change': ['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism'], 'Self-enhancement': ['Achievement', 'Power: dominance', 'Power: resources', 'Face'], 'Conservation': ['Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal'], 'Self-transcendence': ['Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance']}


In [154]:
# Assuming column_names is a list containing all the column names
column_names = [
    "Self-direction: thought",
    "Self-direction: action",
    "Stimulation",
    "Hedonism",
    "Achievement",
    "Power: dominance",
    "Power: resources",
    "Face",
    "Security: personal",
    "Security: societal",
    "Tradition",
    "Conformity: rules",
    "Conformity: interpersonal",
    "Humility",
    "Benevolence: caring",
    "Benevolence: dependability",
    "Universalism: concern",
    "Universalism: nature",
    "Universalism: tolerance"
]
# Define category mappings
category_mapping = {
    "Openness to change": column_names[:4],
    "Self-enhancement": column_names[4:8],
    "Conservation": column_names[8:13],
    "Self-transcendence": column_names[13:]
}

# Reverse the mappings
reverse_mapping = {col: category for category, cols in category_mapping.items() for col in cols}

print(reverse_mapping)


{'Self-direction: thought': 'Openness to change', 'Self-direction: action': 'Openness to change', 'Stimulation': 'Openness to change', 'Hedonism': 'Openness to change', 'Achievement': 'Self-enhancement', 'Power: dominance': 'Self-enhancement', 'Power: resources': 'Self-enhancement', 'Face': 'Self-enhancement', 'Security: personal': 'Conservation', 'Security: societal': 'Conservation', 'Tradition': 'Conservation', 'Conformity: rules': 'Conservation', 'Conformity: interpersonal': 'Conservation', 'Humility': 'Self-transcendence', 'Benevolence: caring': 'Self-transcendence', 'Benevolence: dependability': 'Self-transcendence', 'Universalism: concern': 'Self-transcendence', 'Universalism: nature': 'Self-transcendence', 'Universalism: tolerance': 'Self-transcendence'}


In [155]:
s = []

In [156]:
for i in range(len(vals)):
  k = []
  for j in range(0,38,2):
    a = vals[i,j]
    b = vals[i,j+1]

    if a+b >=0.5 :
      k.append(index_to_column[j/2])
  s.append(k)



In [157]:
s

[['Conformity: rules'],
 ['Conformity: rules'],
 ['Hedonism', 'Face', 'Conformity: rules'],
 ['Conformity: rules'],
 ['Conformity: rules'],
 ['Conformity: rules'],
 ['Face', 'Conformity: rules'],
 ['Conformity: rules'],
 [],
 ['Power: dominance', 'Conformity: rules'],
 ['Self-direction: action'],
 ['Self-direction: action'],
 ['Conformity: rules'],
 [],
 [],
 ['Security: personal'],
 ['Conformity: rules'],
 [],
 ['Conformity: rules'],
 ['Universalism: concern'],
 ['Security: societal'],
 ['Self-direction: thought'],
 ['Power: dominance', 'Security: personal'],
 ['Power: dominance', 'Security: personal'],
 ['Conformity: rules'],
 ['Face'],
 ['Universalism: concern'],
 ['Conformity: rules'],
 [],
 [],
 ['Conformity: rules'],
 ['Conformity: rules'],
 ['Conformity: rules'],
 ['Conformity: rules'],
 ['Conformity: rules'],
 [],
 ['Face'],
 ['Face', 'Universalism: concern'],
 ['Conformity: rules'],
 ['Face', 'Conformity: rules'],
 ['Face'],
 ['Face'],
 ['Conformity: rules'],
 ['Hedonism', 'Ac

In [158]:
ss = []

In [159]:
for i in s:
  m = set()
  for j in i:
    m.add(reverse_mapping[j])
  ss.append(m)

In [160]:
sss = []
for i in s:
  m = set()
  for j in i:
    m.add(j)
  sss.append(m)

In [161]:
sss

[{'Conformity: rules'},
 {'Conformity: rules'},
 {'Conformity: rules', 'Face', 'Hedonism'},
 {'Conformity: rules'},
 {'Conformity: rules'},
 {'Conformity: rules'},
 {'Conformity: rules', 'Face'},
 {'Conformity: rules'},
 set(),
 {'Conformity: rules', 'Power: dominance'},
 {'Self-direction: action'},
 {'Self-direction: action'},
 {'Conformity: rules'},
 set(),
 set(),
 {'Security: personal'},
 {'Conformity: rules'},
 set(),
 {'Conformity: rules'},
 {'Universalism: concern'},
 {'Security: societal'},
 {'Self-direction: thought'},
 {'Power: dominance', 'Security: personal'},
 {'Power: dominance', 'Security: personal'},
 {'Conformity: rules'},
 {'Face'},
 {'Universalism: concern'},
 {'Conformity: rules'},
 set(),
 set(),
 {'Conformity: rules'},
 {'Conformity: rules'},
 {'Conformity: rules'},
 {'Conformity: rules'},
 {'Conformity: rules'},
 set(),
 {'Face'},
 {'Face', 'Universalism: concern'},
 {'Conformity: rules'},
 {'Conformity: rules', 'Face'},
 {'Face'},
 {'Face'},
 {'Conformity: rules

In [162]:
dic = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
for i in ss:
    if len(i) not in dic:
        dic[len(i)] = 1
    else:
        dic[len(i)] += 1

print(dic)


{0: 20381, 1: 18490, 2: 1651, 3: 139, 4: 6, 5: 0}


In [163]:
w = []
for i in range(19):
  s = set()
  w.append(s)


In [164]:
for i in sss:
  for j in i:
    for k in i:
      if j != k:
        w[column_to_index[j]].add(column_to_index[k])

In [165]:
w

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18},
 {0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18},
 {0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18},
 {0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18},
 {1, 3, 4, 5, 7, 9, 10, 12, 14, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,

In [166]:
vals = Y.values

In [167]:
vals

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [168]:
cat1 = []
cat2 = []
cat3 = []

In [169]:
for i in vals:
  su = []
  sub = []
  subs = []
  for j in range(0,38,2):
    a = i[j]
    b = i[j+1]
    if a+b >= 0.5 :
      sub.append(index_to_column[j/2])
      su.append(reverse_mapping[index_to_column[j/2]])
      if ( a / ( a + b ) ) >= 0.5 :
        subs.append("Attained")

      else :
        subs.append("Constrained")
  cat1.append(su)
  cat2.append(sub)
  cat3.append(subs)






In [170]:
X

,Text
0,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ..."
1,КВ.
2,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...
3,Вече месеци наред той е обект на проверка за к...
4,Там от години прокуратурата се явява съдружник...
...,...
40662,Kolay kazanç sağlayan ve istihdam öngörmeyen r...
40663,"Vergi tabana yayılacak, yoksul kesimin vergi y..."
40664,Sosyal refahı artıracak destekleyici düzenleme...
40665,Yoksulluktan en fazla etkilenen eğitim düzeyi ...


In [171]:
new_features_df = pd.DataFrame({'Cat1':cat1, 'Cat2': cat2, 'Cat3': cat3})

In [172]:
new_features_df

,Cat1,Cat2,Cat3
0,[Conservation],[Conformity: rules],[Constrained]
1,[Conservation],[Conformity: rules],[Constrained]
2,"[Openness to change, Self-enhancement, Conserv...","[Hedonism, Face, Conformity: rules]","[Attained, Constrained, Constrained]"
3,[Conservation],[Conformity: rules],[Constrained]
4,[Conservation],[Conformity: rules],[Constrained]
...,...,...,...
40662,"[Openness to change, Self-enhancement]","[Stimulation, Power: dominance]","[Attained, Attained]"
40663,[Self-transcendence],[Benevolence: caring],[Attained]
40664,[Self-enhancement],[Achievement],[Attained]
40665,"[Openness to change, Self-transcendence]","[Self-direction: action, Benevolence: caring]","[Attained, Attained]"


In [173]:
list_2d = X.values.tolist()

In [174]:
df = pd.DataFrame(list_2d, columns=['Text'])
df

,Text
0,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ..."
1,КВ.
2,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...
3,Вече месеци наред той е обект на проверка за к...
4,Там от години прокуратурата се явява съдружник...
...,...
40662,Kolay kazanç sağlayan ve istihdam öngörmeyen r...
40663,"Vergi tabana yayılacak, yoksul kesimin vergi y..."
40664,Sosyal refahı artıracak destekleyici düzenleme...
40665,Yoksulluktan en fazla etkilenen eğitim düzeyi ...


In [175]:
X = pd.concat([df, new_features_df], axis=1)


In [176]:
X

,Text,Cat1,Cat2,Cat3
0,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ...",[Conservation],[Conformity: rules],[Constrained]
1,КВ.,[Conservation],[Conformity: rules],[Constrained]
2,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...,"[Openness to change, Self-enhancement, Conserv...","[Hedonism, Face, Conformity: rules]","[Attained, Constrained, Constrained]"
3,Вече месеци наред той е обект на проверка за к...,[Conservation],[Conformity: rules],[Constrained]
4,Там от години прокуратурата се явява съдружник...,[Conservation],[Conformity: rules],[Constrained]
...,...,...,...,...
40662,Kolay kazanç sağlayan ve istihdam öngörmeyen r...,"[Openness to change, Self-enhancement]","[Stimulation, Power: dominance]","[Attained, Attained]"
40663,"Vergi tabana yayılacak, yoksul kesimin vergi y...",[Self-transcendence],[Benevolence: caring],[Attained]
40664,Sosyal refahı artıracak destekleyici düzenleme...,[Self-enhancement],[Achievement],[Attained]
40665,Yoksulluktan en fazla etkilenen eğitim düzeyi ...,"[Openness to change, Self-transcendence]","[Self-direction: action, Benevolence: caring]","[Attained, Attained]"


In [178]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')




In [179]:
x = X["Text"].tolist()
x = model.encode(x)
y = X[["Cat1","Cat2","Cat3"]].values.tolist()


In [180]:
y

[[['Conservation'], ['Conformity: rules'], ['Constrained']],
 [['Conservation'], ['Conformity: rules'], ['Constrained']],
 [['Openness to change', 'Self-enhancement', 'Conservation'],
  ['Hedonism', 'Face', 'Conformity: rules'],
  ['Attained', 'Constrained', 'Constrained']],
 [['Conservation'], ['Conformity: rules'], ['Constrained']],
 [['Conservation'], ['Conformity: rules'], ['Constrained']],
 [['Conservation'], ['Conformity: rules'], ['Constrained']],
 [['Self-enhancement', 'Conservation'],
  ['Face', 'Conformity: rules'],
  ['Constrained', 'Constrained']],
 [['Conservation'], ['Conformity: rules'], ['Constrained']],
 [[], [], []],
 [['Self-enhancement', 'Conservation'],
  ['Power: dominance', 'Conformity: rules'],
  ['Attained', 'Attained']],
 [['Openness to change'], ['Self-direction: action'], ['Attained']],
 [['Openness to change'], ['Self-direction: action'], ['Constrained']],
 [['Conservation'], ['Conformity: rules'], ['Attained']],
 [[], [], []],
 [[], [], []],
 [['Conservati

In [181]:
z = []
for i in y:
    l = len(i[0])  # Get the length of the first sublist
    w = []
    for k in range(l):
        sub = []
        for j in range(3):  # Assuming there are always 3 sublists in each element of y
            sub.append(i[j][k])  # Access elements using separate indices
        w.append(sub)
    z.append(w)

y = z
y = np.array(y, dtype=object)

In [182]:
y

array([list([['Conservation', 'Conformity: rules', 'Constrained']]),
       list([['Conservation', 'Conformity: rules', 'Constrained']]),
       list([['Openness to change', 'Hedonism', 'Attained'], ['Self-enhancement', 'Face', 'Constrained'], ['Conservation', 'Conformity: rules', 'Constrained']]),
       ..., list([['Self-enhancement', 'Achievement', 'Attained']]),
       list([['Openness to change', 'Self-direction: action', 'Attained'], ['Self-transcendence', 'Benevolence: caring', 'Attained']]),
       list([['Self-transcendence', 'Benevolence: caring', 'Attained']])],
      dtype=object)

In [183]:
len(x[0])

768

In [184]:
pip install hiclass

In [185]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from hiclass.MultiLabelLocalClassifierPerNode import MultiLabelLocalClassifierPerNode

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

#logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
#knn_classifier = KNeighborsClassifier(n_neighbors=5)
#gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
#dt_classifier = DecisionTreeClassifier(random_state=42)
#rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

x_train,y_train = x, y
svm_classifier = SVC(probability=True,class_weight='balanced',kernel='rbf')


classifier = MultiLabelLocalClassifierPerNode(local_classifier=svm_classifier)

classifier.fit(x_train, y_train)



MultiLabelLocalClassifierPerNode(local_classifier=SVC(class_weight='balanced',
                                                      probability=True))

In [186]:
sentences_df = pd.read_csv('/content/drive/MyDrive/New folder/validation/sentences.tsv', sep='\t')
labels_df = pd.read_csv('/content/drive/MyDrive/New folder/validation/labels.tsv', sep='\t')
df1 = sentences_df
df2 = labels_df
merged_df = pd.merge(df1, df2, on=['Text-ID', 'Sentence-ID'])
merged_df = merged_df
merged_df = merged_df[~merged_df.isin(['0.5', '', pd.NA]).any(axis=1)]

merged_df = merged_df.drop(columns=merged_df.columns[:2])
X = merged_df.iloc[:, :1]  # Assuming the first two columns are features
Y = merged_df.iloc[:, 1:]  # Assuming the rest of the columns are labels

vals = Y.values
cat1 = []
cat2 = []
cat3 = []
for i in vals:
  su = []
  sub = []
  subs = []
  for j in range(0,38,2):
    a = i[j]
    b = i[j+1]
    if a+b >= 0.5 :
      sub.append(index_to_column[j/2])
      su.append(reverse_mapping[index_to_column[j/2]])
      if ( a / ( a + b ) ) >= 0.5 :
        subs.append("Attained")

      else :
        subs.append("Constrained")
  cat1.append(su)
  cat2.append(sub)
  cat3.append(subs)
new_features_df = pd.DataFrame({'Cat1':cat1, 'Cat2': cat2, 'Cat3': cat3})
list_2d = X.values.tolist()
df = pd.DataFrame(list_2d, columns=['Text'])
X = pd.concat([df, new_features_df], axis=1)
x = X["Text"].tolist()
x = model.encode(x)
y = X[["Cat1","Cat2","Cat3"]].values.tolist()

z = []
for i in y:
    l = len(i[0])  # Get the length of the first sublist
    w = []
    for k in range(l):
        sub = []
        for j in range(3):  # Assuming there are always 3 sublists in each element of y
            sub.append(i[j][k])  # Access elements using separate indices
        w.append(sub)
    z.append(w)

y = z
y = np.array(y, dtype=object)







In [187]:
predictions = classifier.predict(x)

In [188]:
LL = len(predictions)

In [189]:
arraya = np.zeros((LL, 38))

In [190]:
arrayp = np.zeros((LL, 38))

In [191]:
for i in range(LL):
  ou = predictions[i]
  for j in ou:
    cl = j[1]
    di = j[2]
    #print(cl,di)
    if cl != '' and di != '':
      idx = column_to_index[cl]
      if di == "Attained":
        idxd = 2*idx
        arrayp[i][idxd] = 1
      else :
        idxd = 2*idx + 1
        arrayp[i][idxd] = 1


In [192]:
for i in range(LL):
  ou = y[i]

  for j in ou:
    cl = j[1]
    di = j[2]
    #print(cl,di)
    if cl != '' and di != '':
      idx = column_to_index[cl]
      if di == "Attained":
        idxd = 2*idx
        arraya[i][idxd] = 1
      else :
        idxd = 2*idx + 1
        arraya[i][idxd] = 1


In [193]:
from sklearn.metrics import classification_report
report = classification_report(arraya, arrayp)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       125
           1       0.00      0.00      0.00        27
           2       0.05      0.05      0.05       320
           3       0.29      0.02      0.04        88
           4       0.12      0.05      0.07       323
           5       0.00      0.00      0.00        34
           6       0.00      0.00      0.00        72
           7       0.00      0.00      0.00        27
           8       0.09      0.36      0.14       666
           9       0.10      0.20      0.14       202
          10       0.10      0.23      0.14       429
          11       0.00      0.00      0.00       109
          12       0.13      0.24      0.17       479
          13       0.00      0.00      0.00       179
          14       0.00      0.00      0.00       121
          15       0.06      0.01      0.01       131
          16       0.00      0.00      0.00        76
    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [194]:
arraya = np.zeros((LL, 19))
arrayp = np.zeros((LL, 19))
for i in range(LL):
  ou = predictions[i]
  for j in ou:
    cl = j[1]
    di = j[2]
    #print(cl,di)
    if cl != '' and di != '':
      idx = column_to_index[cl]
      if di == "Attained":
        idxd = idx
        arrayp[i][idxd] = 1
      else :
        idxd = idx
        arrayp[i][idxd] = 1
for i in range(LL):
  ou = y[i]
  for j in ou:
    cl = j[1]
    di = j[2]
    #print(cl,di)
    if cl != '' and di != '':
      idx = column_to_index[cl]
      if di == "Attained":
        idxd = idx
        arraya[i][idxd] = 1
      else :
        idxd = idx
        arraya[i][idxd] = 1
from sklearn.metrics import classification_report
report = classification_report(arraya, arrayp)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       152
           1       0.07      0.05      0.05       408
           2       0.12      0.04      0.06       357
           3       0.00      0.00      0.00        99
           4       0.12      0.40      0.18       868
           5       0.12      0.22      0.16       538
           6       0.18      0.25      0.21       658
           7       0.07      0.01      0.01       252
           8       0.07      0.14      0.09       249
           9       0.13      0.48      0.20      1105
          10       0.11      0.04      0.05       222
          11       0.13      0.28      0.18       836
          12       0.16      0.02      0.03       181
          13       0.00      0.00      0.00        37
          14       0.13      0.13      0.13       326
          15       0.11      0.03      0.04       260
          16       0.13      0.21      0.16       569
    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
